In [11]:
import pandas as pd
import numpy as np
import warnings
import os
warnings.filterwarnings('ignore')

# DistilBERT

In [1]:
! pip install --quiet lightning
! pip install --quiet transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 841.5/841.5 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.2/802.2 kB 21.2 MB/s eta 0:00:00


In [2]:
import pandas as pd
# importing all necessary packages
from os import listdir
from os.path import join
from sklearn.model_selection import train_test_split
import string
from torch.utils.data import Dataset, DataLoader
import torch
import torchmetrics
import pytorch_lightning as pl
from transformers import AutoModelForSequenceClassification
from pytorch_lightning.loggers import TensorBoardLogger, WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint
from transformers import DistilBertTokenizer

#### Dataset Preparation

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#show train dataframe
train_data_preprocessed = pd.read_csv('/content/drive/MyDrive/NUS_MSBA/BT5153_Final Group Project_Shared Folder/Data/train_preprocessed_v2.csv')
train_data_preprocessed.head()

,text,label,%unique_word_total,%stop_word_total,mean_word_length,mean_char_count_per_word
0,school homework clubs become increasingly popu...,1.0,0.533742,0.411043,5.294479,6.294479
1,widely accepted knowledge great source power s...,1.0,0.491289,0.463415,5.125436,6.128920
2,first impressions great power shape interactio...,1.0,0.617647,0.475490,5.137255,6.137255
3,name address city state zip code email address...,1.0,0.432990,0.417526,5.395189,6.393471
4,limiting car usage numerous advantages benefit...,1.0,0.464876,0.409091,5.396694,6.394628


In [5]:
train_data_preprocessed.shape

(28738, 6)

In [6]:
#show test dataframe
test_data_preprocessed = pd.read_csv('/content/drive/MyDrive/NUS_MSBA/BT5153_Final Group Project_Shared Folder/Data/new_essay_val_preprocessed.csv')
test_data_preprocessed.head()

,text,label,%unique_word_total,%stop_word_total,mean_word_length,mean_char_count_per_word
0,marshall plan test progress test progress whet...,0,0.444112,0.392216,5.384232,6.384232
1,promoting global regional security postcold wa...,0,0.414807,0.424949,5.150101,6.150101
2,womanhood peacemaking taking advantage unity c...,0,0.475947,0.378710,5.583419,6.583419
3,human rights universal western construct cruci...,0,0.370482,0.503012,4.875502,5.875502
4,challenges american foreign service rebuilding...,0,0.467425,0.379524,5.668046,6.668046


In [7]:
#split into train validation and test sets
from sklearn.model_selection import train_test_split

df_train, df_val = train_test_split(train_data_preprocessed, test_size=0.3, random_state=42)
df_test = test_data_preprocessed # rename

print(df_train.shape)
print(df_val.shape)
print(df_test.shape)
df_train.dropna(inplace = True)
df_val.dropna(inplace = True)
df_test.dropna(inplace = True)
print(df_train.shape)
print(df_val.shape)
print(df_test.shape)

(20116, 6)
(8622, 6)
(40, 6)
(20116, 6)
(8621, 6)
(40, 6)


In [8]:
# Prepare for DataLoader and Dataset without converting labels to ordinal
X_train_token_list = []
X_val_token_list = []
X_test_token_list = []
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Process the data
for i in df_train["text"]:
    X_train_token_list.append(tokenizer(i, truncation=True, padding='max_length', max_length=512, return_tensors='pt'))
for i in df_val["text"]:
    X_val_token_list.append(tokenizer(i, truncation=True, padding='max_length', max_length=512, return_tensors='pt'))
for i in df_test["text"]:
    X_test_token_list.append(tokenizer(i, truncation=True, padding='max_length', max_length=512, return_tensors='pt'))


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [9]:
# add labels to token_list for use later as dataset
train_labels = []
for i in df_train["label"]:
  if int(i) == 0:
    train_labels.append(0)
  else:
    train_labels.append(1)
val_labels = []
for i in df_val["label"]:
  if int(i) == 0:
    val_labels.append(0)
  else:
    val_labels.append(1)
test_labels = []
for i in df_test["label"]:
  if int(i) == 0:
    test_labels.append(0)
  else:
    test_labels.append(1)
for i in range(len(X_train_token_list)):
  X_train_token_list[i]["label"] = torch.tensor(train_labels[i])
for i in range(len(X_val_token_list)):
  X_val_token_list[i]["label"] = torch.tensor(val_labels[i])
for i in range(len(X_test_token_list)):
  X_test_token_list[i]["label"] = torch.tensor(test_labels[i])


In [12]:
# add additoinal features to token_list for use later as dataset
for i in range(len(X_train_token_list)):
    row = df_train.iloc[i]
    X_train_token_list[i]['%unique_word_total'] = row['%unique_word_total'].astype(np.float32)
    X_train_token_list[i]['%stop_word_total'] = row['%stop_word_total'].astype(np.float32)
    X_train_token_list[i]['mean_word_length'] = row['mean_word_length'].astype(np.float32)
    X_train_token_list[i]['mean_char_count_per_word'] = row['mean_char_count_per_word'].astype(np.float32)

for i in range(len(X_val_token_list)):
    row = df_val.iloc[i]
    X_val_token_list[i]['%unique_word_total'] = row['%unique_word_total'].astype(np.float32)
    X_val_token_list[i]['%stop_word_total'] = row['%stop_word_total'].astype(np.float32)
    X_val_token_list[i]['mean_word_length'] = row['mean_word_length'].astype(np.float32)
    X_val_token_list[i]['mean_char_count_per_word'] = row['mean_char_count_per_word'].astype(np.float32)

for i in range(len(X_test_token_list)):
    row = df_test.iloc[i]
    X_test_token_list[i]['%unique_word_total'] = row['%unique_word_total'].astype(np.float32)
    X_test_token_list[i]['%stop_word_total'] = row['%stop_word_total'].astype(np.float32)
    X_test_token_list[i]['mean_word_length'] = row['mean_word_length'].astype(np.float32)
    X_test_token_list[i]['mean_char_count_per_word'] = row['mean_char_count_per_word'].astype(np.float32)


#### Model Finetuning

In [13]:
!pip install --quiet torch torchmetrics pytorch_lightning transformers

In [14]:
# model finetuning
import torch
import torch.nn as nn
import pytorch_lightning as pl
import torchmetrics
from transformers import AutoModelForSequenceClassification

# train our model and unfreeze our parameters
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2)

for param in model.parameters():
    param.requires_grad = False

for param in model.pre_classifier.parameters():
    param.requires_grad = True

for param in model.classifier.parameters():
    param.requires_grad = True

additional_features_size = 4

# model structure
class LightningModel(pl.LightningModule):
    def __init__(self, model, learning_rate=5e-5):
        super().__init__()

        self.learning_rate = learning_rate

        if model is None:
            raise ValueError("A model must be provided for initialization.")

        self.model = model

        # Ensure the config is set
        self.config = model.config  # Copy the model's configuration

        # New dense layer to process additional features
        self.additional_layer = nn.Linear(additional_features_size, 2)  # Adjust size based on additional features
        self.additional_dropout = nn.Dropout(0.4)

        # Adjusting the classifier to consider additional features
        self.combined_classifier = nn.Linear(2 + 2, 2)  # Assuming binary classification     model.config.hidden_size=768 but bert_logits is 2, so we use 2

        self.train_acc = torchmetrics.Accuracy(task="multiclass", num_classes=2)
        self.train_auroc = torchmetrics.AUROC(task="multiclass", num_classes=2)  # AUROC for training

        self.val_acc = torchmetrics.Accuracy(task="multiclass", num_classes=2)
        self.val_auroc = torchmetrics.AUROC(task="multiclass", num_classes=2)  # AUROC for evaluation
        self.test_acc = torchmetrics.Accuracy(task="multiclass", num_classes=2)
        self.test_auroc = torchmetrics.AUROC(task="multiclass", num_classes=2)  # AUROC for testing


    def forward(self, input_ids, attention_mask, unique_word_total, stop_word_total, mean_word_length, mean_char_count_per_word, labels=None):

        bert_output = self.model(input_ids, attention_mask=attention_mask, labels=labels)
        bert_logits = bert_output["logits"]
        # print('bert_logits')
        # print(bert_logits.shape)
        # print(bert_logits)

        additional_features = torch.stack([unique_word_total, stop_word_total, mean_word_length, mean_char_count_per_word], dim=1)  # Ensure correct shape
        # print(additional_features)
        additional_processed = self.additional_layer(additional_features)
        additional_processed = self.additional_dropout(additional_processed)

        # Combine BERT outputs with additional features
        combined_input = torch.cat([bert_logits, additional_processed], dim=1)

        # Pass combined data through the classifier
        # print(combined_input.shape)
        # print(combined_input)
        combined_logits = self.combined_classifier(combined_input)

        loss = None
        if labels is not None:
            loss = nn.CrossEntropyLoss()(combined_logits, labels)

        return {"logits": combined_logits, "loss": loss}

        # return self.model(input_ids, attention_mask=attention_mask, labels=labels)

    def training_step(self, batch, batch_idx):
        outputs = self(
            input_ids=batch["input_ids"].squeeze(1),
            attention_mask=batch["attention_mask"],
            unique_word_total=batch["%unique_word_total"],
            stop_word_total=batch["%stop_word_total"],
            mean_word_length=batch["mean_word_length"],
            mean_char_count_per_word=batch["mean_char_count_per_word"],
            labels=batch["label"],
        )
        self.log("train_loss", outputs["loss"])
        logits = outputs["logits"]
        probabilities = torch.softmax(logits, dim=1)  # Get probabilities for the positive class
        predicted_labels = torch.argmax(logits, 1)

        # Log accuracy
        self.train_acc(predicted_labels, batch["label"])
        self.log("train_acc", self.train_acc)

        # Update and log AUROC for training
        self.train_auroc(probabilities, batch["label"])
        # self.train_auroc.update(probabilities[:, 1], batch["label"])  # Use probabilities for the positive class
        self.log("train_auroc", self.train_auroc)

        return outputs["loss"]



    def validation_step(self, batch, batch_idx):
        outputs = self(
            input_ids=batch["input_ids"].squeeze(1),
            attention_mask=batch["attention_mask"],
            unique_word_total=batch["%unique_word_total"],
            stop_word_total=batch["%stop_word_total"],
            mean_word_length=batch["mean_word_length"],
            mean_char_count_per_word=batch["mean_char_count_per_word"],
            labels=batch["label"],
        )
        self.log("val_loss", outputs["loss"], prog_bar=True)

        logits = outputs["logits"]
        probabilities = torch.softmax(logits, dim=1)  # Get probabilities
        predicted_labels = torch.argmax(logits, 1)

        # Log accuracy
        self.val_acc(predicted_labels, batch["label"])
        self.log("val_acc", self.val_acc, prog_bar=True)

        # Update and log AUROC
        self.val_auroc(probabilities, batch["label"])
        # self.val_auroc.update(probabilities[:, 1], batch["label"])  # Use probabilities for the positive class
        self.log("val_auroc", self.val_auroc, prog_bar=True)

        # output for classification report
        return {"logits": logits, "loss": outputs["loss"], "labels": batch["label"]}


    def test_step(self, batch, batch_idx):
        outputs = self(
            input_ids=batch["input_ids"].squeeze(1),
            attention_mask=batch["attention_mask"],
            unique_word_total=batch["%unique_word_total"],
            stop_word_total=batch["%stop_word_total"],
            mean_word_length=batch["mean_word_length"],
            mean_char_count_per_word=batch["mean_char_count_per_word"],
            labels=batch["label"],
        )

        logits = outputs["logits"]
        probabilities = torch.softmax(logits, dim=1)  # Get probabilities
        predicted_labels = torch.argmax(logits, 1)

        # Log accuracy
        self.test_acc(predicted_labels, batch["label"])
        self.log("test_acc", self.test_acc, prog_bar=True)

        # Update and log AUROC
        self.test_auroc(probabilities, batch["label"])
        self.log("test_auroc", self.test_auroc, prog_bar=True)

        # output for classification report
        return {"logits": logits, "loss": outputs["loss"], "labels": batch["label"]}


    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer


lightning_model = LightningModel(model)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
# Create dataset and dataloader
class MyDataset(Dataset):
    def __init__(self, array_of_dicts):
        self.data = array_of_dicts

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        return item

trainset = MyDataset(X_train_token_list)
valset = MyDataset(X_val_token_list)
testset = MyDataset(X_test_token_list)

train_loader = DataLoader(
    dataset=trainset,
    batch_size=4,
    shuffle=True,
    num_workers=0
)

val_loader = DataLoader(
    dataset=valset,
    batch_size=4,
    num_workers=0
)

test_loader = DataLoader(
    dataset=testset,
    batch_size=4,
    num_workers=0
)

#### Model Training

##### Monitor val_auroc

In [16]:
# Change the monitor to "val_auroc"
logger = TensorBoardLogger("distilbert-add-fea/", name="finetuning", version="original")
callbacks = [
    ModelCheckpoint(
        save_top_k=1, mode="max", monitor="val_auroc"  # Monitor the AUROC metric
    )
]

In [17]:
# train model
trainer = pl.Trainer(
    max_epochs=3,
    callbacks=callbacks,
    # accelerator="gpu",  uncomment if gpu available
    devices=1,
    logger=logger,
    log_every_n_steps=1,
)

# Start training
trainer.fit(
    model=lightning_model,
    train_dataloaders=train_loader,
    val_dataloaders=val_loader
)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name                | Type                                | Params
----------------------------------------------------------------------------
0 | model               | DistilBertForSequenceClassification | 67.0 M
1 | additional_layer    | Linear                              | 10    
2 | additional_dropout  | Dropout                             | 0     
3 | combined_classifier | Linear                              | 10    
4 | train_acc           | MulticlassAccuracy                  | 0     
5 | train_auroc         | MulticlassAUROC 

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=3` reached.


In [18]:
# Define the path to save the model
model_path = "/content/drive/MyDrive/NUS_MSBA/BT5153_Final Group Project_Shared Folder/models/distilbert/latest/distilbert-dataset-v2_metafea_ep3_dropout0.4.pth"

# Save the entire model
torch.save(lightning_model, model_path)

In [ ]:
# # Define the path to the saved model
# model_path = "distilbert-meta-fea_epoch2.pth"

# # Load the model
# loaded_model = torch.load(model_path)

# print("Model loaded.")

In [19]:
# test with validation dataset
val_results = trainer.validate(lightning_model, dataloaders=val_loader, ckpt_path="best")
val_results

INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at distilbert-add-fea/finetuning/original/checkpoints/epoch=2-step=15087.ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from the checkpoint at distilbert-add-fea/finetuning/original/checkpoints/epoch=2-step=15087.ckpt


Validation: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│          val_acc          │    0.9181069731712341     │
│         val_auroc         │    0.9742077589035034     │
│         val_loss          │    0.20640021562576294    │
└───────────────────────────┴───────────────────────────┘

[{'val_loss': 0.20640021562576294,
  'val_acc': 0.9181069731712341,
  'val_auroc': 0.9742077589035034}]

#### Test data

In [20]:
# test with final unseen test dataset
test_results = trainer.test(lightning_model, dataloaders=test_loader, ckpt_path="best")
test_results

INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at distilbert-add-fea/finetuning/original/checkpoints/epoch=2-step=15087.ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from the checkpoint at distilbert-add-fea/finetuning/original/checkpoints/epoch=2-step=15087.ckpt


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │     0.800000011920929     │
│        test_auroc         │           0.875           │
└───────────────────────────┴───────────────────────────┘

[{'test_acc': 0.800000011920929, 'test_auroc': 0.875}]

#### Tensor Board

In [ ]:
# Start tensorboard.
%load_ext tensorboard
%tensorboard --logdir distilbert/finetuning/original/

Reusing TensorBoard on port 6006 (pid 71933), started 1:41:39 ago. (Use '!kill 71933' to kill it.)